## Imports and Data Preprocessing

Import data and do basics of removing extraneous data

In [1]:
# Initial imports
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [2]:
# Import Verlander dataset 

verlander_df = pd.read_csv(Path("../resources/verlander_update.csv"),
                            index_col='game_date',
                            parse_dates=True,
                            infer_datetime_format=True)

display(verlander_df.head())

,pitch_type,player_name,batter,pitcher,events,description,zone,des,stand,p_throws,...,on_1b,outs_when_up,inning,at_bat_number,pitch_number,pitch_name,bat_score,fld_score,if_fielding_alignment,of_fielding_alignment
game_date,,,,,,,,,,,,,,,,,,,,,
2022-09-16,SL,"Verlander, Justin",669127,434378,strikeout,swinging_strike,9,Shea Langeliers strikes out swinging.,R,R,...,NaN,2,5,35,5,Slider,0,2,Infield shift,Standard
2022-09-16,FF,"Verlander, Justin",669127,434378,NaN,foul,3,Shea Langeliers strikes out swinging.,R,R,...,NaN,2,5,35,4,4-Seam Fastball,0,2,Infield shift,Standard
2022-09-16,SL,"Verlander, Justin",669127,434378,NaN,called_strike,13,Shea Langeliers strikes out swinging.,R,R,...,NaN,2,5,35,3,Slider,0,2,Infield shift,Standard
2022-09-16,FF,"Verlander, Justin",669127,434378,NaN,foul,1,Shea Langeliers strikes out swinging.,R,R,...,NaN,2,5,35,2,4-Seam Fastball,0,2,Infield shift,Standard
2022-09-16,SL,"Verlander, Justin",669127,434378,NaN,ball,8,Shea Langeliers strikes out swinging.,R,R,...,NaN,2,5,35,1,Slider,0,2,Infield shift,Standard


In [3]:
# Clean dataset 

verlander_df = verlander_df.drop(columns=['des', 
'pitch_name', 
'at_bat_number', 
'inning', 
'zone', 
'type', 
'player_name', 
'batter', 
'pitcher', 
'events',
'bb_type',
'hit_location'])

display(verlander_df.head())

,pitch_type,description,stand,p_throws,home_team,away_team,balls,strikes,on_3b,on_2b,on_1b,outs_when_up,pitch_number,bat_score,fld_score,if_fielding_alignment,of_fielding_alignment
game_date,,,,,,,,,,,,,,,,,
2022-09-16,SL,swinging_strike,R,R,HOU,OAK,1,2,NaN,NaN,NaN,2,5,0,2,Infield shift,Standard
2022-09-16,FF,foul,R,R,HOU,OAK,1,2,NaN,NaN,NaN,2,4,0,2,Infield shift,Standard
2022-09-16,SL,called_strike,R,R,HOU,OAK,1,1,NaN,NaN,NaN,2,3,0,2,Infield shift,Standard
2022-09-16,FF,foul,R,R,HOU,OAK,1,0,NaN,NaN,NaN,2,2,0,2,Infield shift,Standard
2022-09-16,SL,ball,R,R,HOU,OAK,0,0,NaN,NaN,NaN,2,1,0,2,Infield shift,Standard


In [4]:
# Encode 1st, 2nd and 3rd bases with 1s and 0s

onbase_df = verlander_df[['on_1b', 'on_2b', 'on_3b']]

onbase_df.loc[onbase_df['on_1b']==np.nan] = 0
onbase_df.loc[onbase_df['on_2b']==np.nan] = 0
onbase_df.loc[onbase_df['on_3b']==np.nan] = 0

onbase_df.loc[onbase_df['on_1b']>0] = 1
onbase_df.loc[onbase_df['on_2b']>0] = 1
onbase_df.loc[onbase_df['on_3b']>0] = 1

# for label, content in onbase_df.iterrows():
#     label.replace(np.nan, 0)
# print("phew finally finished")

#onbase_df['on_1b'].value_counts()
display(onbase_df[1220:1400])

/var/folders/zx/3xfmqcd54f3fdz2hfpnll0_80000gn/T/ipykernel_1571/3735095843.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onbase_df.loc[onbase_df['on_1b']==np.nan] = 0
/var/folders/zx/3xfmqcd54f3fdz2hfpnll0_80000gn/T/ipykernel_1571/3735095843.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onbase_df.loc[onbase_df['on_2b']==np.nan] = 0
/var/folders/zx/3xfmqcd54f3fdz2hfpnll0_80000gn/T/ipykernel_1571/3735095843.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

,on_1b,on_2b,on_3b
game_date,,,
2022-06-12,1.0,1.0,1.0
2022-06-12,1.0,1.0,1.0
2022-06-12,1.0,1.0,1.0
2022-06-12,1.0,1.0,1.0
2022-06-12,1.0,1.0,1.0
...,...,...,...
2022-06-07,NaN,NaN,NaN
2022-06-07,NaN,NaN,NaN
2022-06-07,NaN,NaN,NaN


## Data Processing 

Prepare data to be fed into model

In [5]:
# Split into X and y 

X = verlander_df.drop(columns='pitch_type')
y= verlander_df['pitch_type']

In [6]:
# Use get_dummies to encode categorical variables 

X = pd.get_dummies(X)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

## Deep Learning/Neural Network Architecture

Using the method from the module, the following basis is used to design the first iteration of the neural network: 

*the mean of the number of input features and the number of neurons in the output layer ((number of input features + number of neurons in output layer) / 2). Use a number close to this mean for the number of neurons in the first hidden layer. Repeat this pattern for subsequent hidden layers ((number of neurons in the prior hidden layer + number of neurons in output layer) / 2). Softmax is the activation for the output layer that is used for multi-class classification. Categorial cross entropy and predictive model accuracy are respectively the loss functions and metrics used for multi-class classification*

In [7]:
# Initialize the Deep Learning Neural Network model

nn_v0 = Sequential()

2022-09-22 09:42:21.396243: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
# Design the network architecture 

# Define the model - deep neural net
number_input_features = len(verlander_df.columns)
number_output = 1

# Define hidden layers
i = 0
hidden_nodes_layer=(number_input_features+number_output)/2
while hidden_nodes_layer > 1: 
    if i == 0:
        nn_v0.add(Dense(units=round(hidden_nodes_layer), input_dim=number_input_features, activation='relu'))
        i+=1
    else:
        hidden_nodes_layer = hidden_nodes_layer/2
        nn_v0.add(Dense(units=round(hidden_nodes_layer), activation='relu'))
        i+=1

# Define output layer
nn_v0.add(Dense(units=number_output, activation='softmax'))

# Compile the model
nn_v0.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

display(len(verlander_df.columns))
display(nn_v0.summary())

17

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 9)                 162       
                                                                 
 dense_1 (Dense)             (None, 4)                 40        
                                                                 
 dense_2 (Dense)             (None, 2)                 10        
                                                                 
 dense_3 (Dense)             (None, 1)                 3         
                                                                 
 dense_4 (Dense)             (None, 1)                 2         
                                                                 
 dense_5 (Dense)             (None, 1)                 2         
                                                                 
 dense_6 (Dense)             (None, 9)                 1

None